<a href="https://colab.research.google.com/github/londhevishalps/sustainability-news/blob/main/Sustainability_updates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feedparser newspaper3k sentence-transformers openai pandas scikit-learn flask pyngrok nltk
from google.colab import drive
drive.mount('/content/drive')


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 840.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 4.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=c792d17a67d0a97736bb9feeca3bb0204d4b414a8b0b34d9a53ae72de357310c
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=6cad95af2c9d9186723fff91d2f4459808dd715c61b166a8c0af57c3a20905fa
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f1

In [2]:
# Step 1 — RSS + BeautifulSoup scraping (expanded sources)
!pip install feedparser beautifulsoup4 pandas

import feedparser
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import json

# --- List of RSS feeds ---
sources = {
    "Sustainability Mag": "https://sustainabilitymag.com/feed/",
    "Environmental News Network": "https://www.enn.com/rss/",
    "Science Daily - Environment": "https://www.sciencedaily.com/rss/earth_climate/environment.xml",
    "New Scientist - Environment": "https://www.newscientist.com/subject/environment/rss/",
    "ESG Daily": "https://esgdaily.com/feed/",
    "Edie.net": "https://www.edie.net/rss/news/",
    "Business Green": "https://www.businessgreen.com/rss/news",
    "Sustainable Procurement News": "https://www.sustainableprocurementnews.com/feed/",
    "Recharge News": "https://www.rechargenews.com/rss",
    "GreenBiz": "https://www.greenbiz.com/feeds/news",
    # New sources
    "BBC Environment": "http://feeds.bbci.co.uk/news/science_and_environment/rss.xml",
    "Al Jazeera Environment": "https://www.aljazeera.com/xml/rss/all.xml",
    "CNN Environment": "http://rss.cnn.com/rss/edition_world.rss",
    "The Guardian Environment": "https://www.theguardian.com/environment/rss",
    "Europe News - Environment": "https://www.euronews.com/rss?level=theme&name=environment"
}

# --- Filter settings ---
max_age_days = 7  # only articles from the last 7 days
today = datetime.now()

articles = []

for source_name, rss_url in sources.items():
    print(f"Fetching {source_name}...")
    feed = feedparser.parse(rss_url)

    for entry in feed.entries:
        # Parse publish date
        if hasattr(entry, 'published_parsed'):
            pub_date = datetime(*entry.published_parsed[:6])
        elif hasattr(entry, 'updated_parsed'):
            pub_date = datetime(*entry.updated_parsed[:6])
        else:
            pub_date = today

        # Skip old articles
        if (today - pub_date).days > max_age_days:
            continue

        # Clean summary text
        summary = getattr(entry, 'summary', '') or ''
        soup = BeautifulSoup(summary, 'html.parser')
        clean_summary = soup.get_text().strip()

        # Store article
        articles.append({
            "title": entry.title,
            "summary": clean_summary,
            "link": entry.link,
            "source": source_name,
            "publish_date": pub_date.strftime("%d-%m-%Y")
        })

print(f"✅ Total recent articles collected: {len(articles)}")

# --- Save to JSON for Step 2 ---
json_path = '/content/drive/MyDrive/sustainability_raw_articles.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)

print(f"Saved raw articles JSON to: {json_path}")


Fetching Sustainability Mag...
Fetching Environmental News Network...
Fetching Science Daily - Environment...
Fetching New Scientist - Environment...
Fetching ESG Daily...
Fetching Edie.net...
Fetching Business Green...
Fetching Sustainable Procurement News...
Fetching Recharge News...
Fetching GreenBiz...
Fetching BBC Environment...
Fetching Al Jazeera Environment...
Fetching CNN Environment...
Fetching The Guardian Environment...
Fetching Europe News - Environment...
✅ Total recent articles collected: 117
Saved raw articles JSON to: /content/drive/MyDrive/sustainability_raw_articles.json


In [3]:
# ✅ STEP 2 – Filter sustainability-related articles (last 14 days)

import json
from datetime import datetime, timedelta
from tqdm import tqdm

# === Configuration ===
raw_file_path = "/content/drive/MyDrive/sustainability_raw_articles.json"
filtered_file_path = "/content/drive/MyDrive/filtered_articles.json"

# Keywords for sustainability topics
sustainability_keywords = [
    "sustainability", "sustainable", "climate", "environment",
    "carbon", "emission", "renewable", "energy", "biodiversity",
    "pollution", "waste", "water", "green", "ecology",
    "circular", "net zero", "deforestation", "plastic", "recycling"
]

# Define time window: last 14 days
cutoff_date = datetime.now() - timedelta(days=14)

# === Load raw articles ===
with open(raw_file_path, "r", encoding="utf-8") as f:
    raw_articles = json.load(f)

print(f"Loaded {len(raw_articles)} raw articles from Step 1")

# === Filter by date + keywords ===
filtered_articles = []

for article in tqdm(raw_articles, desc="Filtering for sustainability topics"):
    publish_date_str = article.get("publish_date", "")
    try:
        publish_date = datetime.strptime(publish_date_str, "%d-%m-%Y")
    except ValueError:
        # Skip if date format is invalid
        continue

    if publish_date >= cutoff_date:
        title = article.get("title", "").lower()
        summary = article.get("summary", "").lower()
        if any(keyword in title or keyword in summary for keyword in sustainability_keywords):
            filtered_articles.append(article)

# === Save filtered results ===
with open(filtered_file_path, "w", encoding="utf-8") as f:
    json.dump(filtered_articles, f, indent=2, ensure_ascii=False)

print(f"✅ {len(filtered_articles)} sustainability-related articles found.")
print(f"💾 Saved filtered sustainability articles to {filtered_file_path}\n")

# === Preview a few ===
if filtered_articles:
    print("🔍 Example filtered article:")
    print(json.dumps(filtered_articles[0], indent=2, ensure_ascii=False))
else:
    print("⚠️ No sustainability-related articles found in the last 14 days.")


Loaded 117 raw articles from Step 1


Filtering for sustainability topics: 100%|██████████| 117/117 [00:00<00:00, 8968.74it/s]


✅ 20 sustainability-related articles found.
💾 Saved filtered sustainability articles to /content/drive/MyDrive/filtered_articles.json

🔍 Example filtered article:
{
  "title": "Water companies told to refund £260m to customers for poor performance",
  "summary": "But bills are still due to rise to fund improvements to water infrastructure.",
  "link": "https://www.bbc.com/news/articles/cdjrymnx1e8o?at_medium=RSS&at_campaign=rss",
  "source": "BBC Environment",
  "publish_date": "23-10-2025"
}


In [4]:
# === STEP 3 — Cluster sustainability articles ===

import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

# --- Load filtered JSON from Step 2 ---
filtered_path = "/content/drive/MyDrive/filtered_articles.json"
with open(filtered_path, "r", encoding="utf-8") as f:
    data = json.load(f)

if not data:
    raise ValueError("No articles found in the JSON file. Check Step 2 results.")

# --- Convert to DataFrame ---
df = pd.DataFrame(data)

# Ensure 'summary' column exists
df["summary"] = df.get("summary", "")
df["summary"] = df["summary"].fillna(df["title"])

# --- Handle small datasets ---
if len(df) < 2:
    print("⚠️ Not enough articles for clustering. Skipping Step 3.")
    df["cluster"] = 0
else:
    # --- TF-IDF vectorization ---
    vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
    X = vectorizer.fit_transform(df["summary"])

    # --- Agglomerative clustering ---
    n_clusters = min(10, len(df))
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        metric="cosine",
        linkage="average"
    )
    df["cluster"] = clustering.fit_predict(X.toarray())
    print("✅ Clustering completed")

# --- Save clustered JSON for Step 4 ---
clustered_path = "/content/drive/MyDrive/clustered_articles.json"
df.to_json(clustered_path, orient="records", force_ascii=False, indent=2)
print(f"💾 Clustered articles saved to {clustered_path}")


✅ Clustering completed
💾 Clustered articles saved to /content/drive/MyDrive/clustered_articles.json


In [5]:
# --- Step 4: Summarize clustered articles with dynamic lengths ---

import json
from transformers import pipeline

# Load clustered articles
clustered_path = "/content/drive/MyDrive/clustered_articles.json"
with open(clustered_path, "r", encoding="utf-8") as f:
    clustered_data = json.load(f)

if not clustered_data:
    raise ValueError("No clustered articles found. Check Step 3 results.")

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)  # use GPU if available
summaries = []

df = pd.DataFrame(clustered_data)

for cluster_id in df["cluster"].unique():
    subset = df[df["cluster"] == cluster_id]

    # Combine titles and summaries
    joined_texts = "\n\n".join(
        f"- {row['title']} ({row['source']})\n{row['summary']}" for _, row in subset.iterrows()
    )

    # Count words in input
    input_word_count = len(joined_texts.split())

    # Dynamically set max_length relative to input size
    # Ensures output is shorter than input, but not too short
    max_len = max(20, min(120, input_word_count // 2))
    min_len = max(10, max_len // 3)

    summary_text = summarizer(
        joined_texts,
        max_length=max_len,
        min_length=min_len,
        do_sample=False
    )[0]["summary_text"]

    summaries.append({
        "cluster_id": int(cluster_id),
        "summary": summary_text,
        "sources": [row["source"] for _, row in subset.iterrows()],
        "articles": subset.to_dict(orient="records")
    })

# Save summaries to JSON
summary_path = "/content/drive/MyDrive/sustainability_summaries.json"
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summaries, f, ensure_ascii=False, indent=2)

print(f"✅ Summarized news saved to JSON: {summary_path}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ Summarized news saved to JSON: /content/drive/MyDrive/sustainability_summaries.json


In [ ]:
!pip install gradio --upgrade

import gradio as gr
import json
from datetime import datetime

# Load summarized JSON
summary_path = "/content/drive/MyDrive/sustainability_summaries.json"
with open(summary_path, "r", encoding="utf-8") as f:
    summaries = json.load(f)

# Function to render news clusters as responsive cards
def show_news():
    html_output = """
    <div style="font-family: Arial, sans-serif; padding: 10px; max-width:1200px; margin:auto;">
        <h1 style="color: #2b7a0b; text-align:center;">🌱 Sustainability News</h1>
        <div style="display: flex; flex-wrap: wrap; gap: 15px; justify-content: center;">
    """
    for cluster in summaries:
        html_output += """
        <div style="flex: 1 1 300px; border:1px solid #ccc; border-radius:12px; padding:15px; background:#f9f9f9; min-width:280px; max-width:450px; box-shadow:0 2px 5px rgba(0,0,0,0.1);">
        """
        # Summary header
        html_output += f"""
            <div style="background:#2b7a0b; color:white; padding:8px 12px; border-radius:8px; margin-bottom:10px; font-weight:bold; font-size:16px;">
                {cluster['summary']}
            </div>
            <ul style="padding-left:20px; margin-top:0;">
        """
        # Articles within cluster
        for article in cluster['articles']:
            try:
                date_str = datetime.strptime(article['publish_date'], "%d-%m-%Y").strftime("%d-%m-%Y")
            except:
                date_str = article['publish_date']
            html_output += f"""
                <li style="margin-bottom:8px; font-size:14px;">
                    <a href="{article['link']}" target="_blank" style="color:#0b5394; text-decoration:none;">{article['title']}</a><br>
                    <span style="font-size:12px; color:#555;">{article['source']} | {date_str}</span>
                </li>
            """
        html_output += "</ul></div>"  # end card
    html_output += "</div></div>"  # end container
    return html_output

# Gradio app with HTML output
iface = gr.Interface(
    fn=show_news,
    inputs=[],
    outputs=gr.HTML(),
    title="🌱 Sustainability News Aggregator",
    description="Aggregated sustainability news with cluster summaries, source links, and publication dates. Curated by Vishal Londhe."
)

# Launch Gradio
iface.launch()
